We have precreated database `stackoverflow_`. Firstly, investigate it. What tables does it has?

# Hive Assignment 2. DML: Find Most Popular Tags
Compare most popular tags in year 2016 with tags in 2009. Tag popularity is the number of questions (post_type_id=1) with this tag. Output top 10 tags in format:



Example:
`tag <tab> rank in 2016 <tab> rank in 2009 <tab> tag popularity in 2016 <tab> tag popularity in 2009`

`php 5 3 1234 5678`


* For 'rank' in each year use rank() function. Order by 'rank in 2016'.



* The part of the result on the sample dataset:
`...
android 3   52  1809    25
php 4   3   1673    215
python  5   11  1585    108
c#  6   1   1519    423
...`




In [29]:
! hive --database stackoverflow_ -e 'show tables'


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 0.309 seconds
OK
posts
users
Time taken: 0.109 seconds, Fetched: 2 row(s)


###### Querying to the database

Add necessary JAR-files.

In [30]:
%%writefile query.hql

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

Overwriting query.hql


Include precreated database `stackoverflow_`.

In [31]:
%%writefile -a query.hql

USE stackoverflow_;
DESCRIBE posts;
DESCRIBE users;


Appending to query.hql


Then, write a simple query.

In [32]:
%%writefile -a query.hql

SELECT * FROM posts LIMIT 10;
SELECT * FROM users LIMIT 10;

Appending to query.hql


And finally, let's execute received `HiveQL`-file.

In [33]:
! hive -f query.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar]
OK
Time taken: 0.29 seconds
OK
id                  	int                 	                    
post_type_id        	tinyint             	                    
date                	string              	                    
owner_user_id       	int                 	                    
parent_id           	int                 	                    
score               	int                 	                    
favorite_count      	int                 	                    
tags                	array<string>       	                    
year                	string 

In [42]:
%%writefile query2.hql
USE stackoverflow_;

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;


SELECT c.token, c.rank, p.rank, c.cnt, p.cnt 
FROM
    (SELECT exploded.year as year, exploded.cnt as cnt, exploded.myCol1 as token, RANK() OVER (ORDER BY exploded.cnt DESC) rank 
    FROM
        (SELECT year, myCol1, COUNT(*) AS cnt FROM posts
            LATERAL VIEW explode(tags) myTable1 AS myCol1
            WHERE post_type_id = 1 AND year = 2016
            GROUP BY year, myCol1) as exploded
    WHERE exploded.year = 2016
    LIMIT 10) AS c
LEFT OUTER JOIN 
    (SELECT exploded.year as year, exploded.cnt as cnt, exploded.myCol1 as token, RANK() OVER (ORDER BY exploded.cnt DESC) rank 
    FROM
        (SELECT year, myCol1, COUNT(*) AS cnt FROM posts
            LATERAL VIEW explode(tags) myTable1 AS myCol1
            WHERE post_type_id = 1 AND year = 2009
            GROUP BY year, myCol1) as exploded
    WHERE exploded.year = 2009) AS p
ON c.token = p.token
ORDER BY c.rank


Overwriting query2.hql


In [43]:

! hive -f query2.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 0.308 seconds
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar]
Query ID = jovyan_20180809235555_369ddc8e-2fd6-4d1f-93e8-0d93e46637b6
Total jobs = 8
Launching Job 1 out of 8
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1533854501399_0018, Tracking URL = http://dab5d

javascript	1	5	2771	192
java	2	2	2033	243
android	3	52	1809	25
php	4	3	1673	215
python	5	11	1585	108
c#	6	1	1519	423
html	7	14	1212	84
jquery	8	8	1167	141
ios	9	186	914	7
css	10	20	801	59
Time taken: 127.208 seconds, Fetched: 10 row(s)


# Alternative solution (Improved readability - by Maxim Uvarov)

In [45]:
%%writefile query.hql

USE stackoverflow_;


WITH questions AS (
    SELECT year, tags_expl, COUNT(*) AS cnt
    FROM posts
    LATERAL VIEW explode(tags) tagsTable AS tags_expl
    WHERE post_type_id = 1 AND year IN (2009, 2016)
    GROUP BY year, tags_expl
)
SELECT concat_ws("\t", tags_expl, string(rank_2016), string(rank_2009), string(cnt_2016), string(cnt_2009))
FROM (
    SELECT
        t_2016.tags_expl AS tags_expl
        ,t_2016.rank AS rank_2016
        ,t_2009.rank AS rank_2009
        ,t_2016.cnt AS cnt_2016
        ,t_2009.cnt AS cnt_2009
    FROM (
        SELECT tags_expl, cnt, RANK() OVER (ORDER BY cnt DESC) rank
        FROM questions
        WHERE year = 2016
        LIMIT 10
    ) t_2016
    LEFT OUTER JOIN (
        SELECT tags_expl, cnt, RANK() OVER (ORDER BY cnt DESC) rank
        FROM questions
        WHERE year = 2009
    ) t_2009
    ON t_2016.tags_expl = t_2009.tags_expl
    ORDER BY rank_2016
) t_top;

Overwriting query.hql
